In [1]:
!pip install -q langchain pypdf sentence-transformers chromadb openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q langchain pypdf sentence-transformers chromadb langchain-openai faiss-gpu --upgrade --quiet  rank_bm25 > /dev/null

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("/content/CNN 기반 공동주택 도배하자 검출 딥러닝 이미지 분류모델 구축에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택 건축공사의 하자유형과 대책에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택 세대내부 결로발생 하자 방지를 위한 단열재 시공 개선방안에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택 하자발생 원인과 방지방안에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택에서 하자담보책임과 하자보수보증책임에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택의 공정별 하자발생 및 예방대책에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택의 목질계 바닥 마감재의 하자 유형 분석 및 대책.pdf"),
    PyPDFLoader("/content/벽지반점_조사연구_내화건축자재협회_최종_20151026.pdf"),
    PyPDFLoader("/content/시대별 공동주택 설계변천에 따른 하자유형 변화 및 특징에 관한 연구.pdf"),
    PyPDFLoader("/content/신축공동주택 실내마감공사 하자 원인분석 및 예방방안.pdf"),

]
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

In [ ]:
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2



embedding = ko_embedding
faiss_vectorstore = FAISS.from_documents(texts, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = '--'

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

openai = ChatOpenAI(model_name="gpt-3.5-turbo", temperature = 0)

qa = RetrievalQA.from_chain_type(llm = openai,
                                 chain_type = "stuff",
                                 retriever = ensemble_retriever,
                                 return_source_documents = True)

def making_query(keywords):
    for keyword in keywords:
        query = "{}에 대한 질문을 가능한 많이 만들어줘".format(keyword)
        result = qa(query)  # Assuming `qa()` is a function defined elsewhere
        print(result['result'])
    return "done"


keywords = []
print(making_query(keywords))
